In [1]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import wandb
import yaml

from models import MomentumBT
from utils import data_helper

In [2]:
config_path = "./config.yaml"
with open(config_path, 'r') as stream:
    config = yaml.safe_load(stream)

In [3]:
pl.seed_everything(config['seed'])

# use a GPU if available
gpus = 1 if torch.cuda.is_available() else 0
device = 'cuda' if gpus else 'cpu'

# Create dataloaders
dataloader_train_ssl, dataloader_train_kNN, dataloader_test = data_helper(config)

# Set up model and training
model = MomentumBT(config, dataloader_train_kNN, gpus=gpus)

Global seed set to 1


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [4]:
check_path = "/home/shkhrmohan94_gmail_com/barlowtwins/wandb/run-20210621_013650-20ssa9h1/files/mlo-ssl/20ssa9h1/checkpoints/epoch=799-step=77599.ckpt"

In [5]:
model = model.load_from_checkpoint(check_path, config=config, dataloader_kNN=dataloader_train_kNN, gpus=gpus)

In [6]:
for batch_idx, batch in enumerate(dataloader_test):
    out = model(batch[0])
    break

In [7]:
(y1, y2), out2 = out

In [8]:
y1_np = y1.detach().cpu().numpy()
y2_np = y2.detach().cpu().numpy()

In [20]:
y2_test.mean(), y2_test.std()

(-9.640644e-11, 0.82242244)

In [26]:
def forward(z_a: torch.Tensor, z_b: torch.Tensor):
        # normalize repr. along the batch dimension
    z_a_norm = (z_a - z_a.mean(0)) / z_a.std(0) # NxD
    z_b_norm = (z_b - z_b.mean(0)) / z_b.std(0) # NxD

    N = z_a.size(0)
    D = z_a.size(1)

        # cross-correlation matrix
    c = torch.mm(z_a_norm.T, z_b_norm) / N # DxD
        # loss
    c_diff = (c - torch.eye(D)).pow(2) # DxD
        # multiply off-diagonal elems of c_diff by lambda
    c_diff[~torch.eye(D, dtype=bool)] *= 5e-3
    loss = c_diff.sum()

    return loss, c

In [27]:
loss,c =  forward(y1, y2)

In [9]:
import swat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from time import time
%matplotlib inline

In [31]:
c_np.sum()

1977.728

In [32]:
c_np.trace()

2044.0